# At the end random forest provide better accuracy than logistic regression

In [114]:
from google.colab import files
uploaded=files.upload()


Saving train.csv to train (1).csv


In [160]:
# IMPORTING THE REQUIRED LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
# Load dataset
df = pd.read_csv('train.csv')

#** imputing the age using the random forest regressor **#

In [144]:

# Drop 'Cabin' and other high-cardinality or less useful columns
df_age = df.drop(['Cabin', 'Name', 'Ticket', 'Embarked'], axis=1)

# Encode 'Sex'
df_age['Sex'] = LabelEncoder().fit_transform(df_age['Sex'])

# Split into rows with and without Age
df_known_age = df_age[df_age['Age'].notna()]
df_unknown_age = df_age[df_age['Age'].isna()]

# Prepare training data
x = df_known_age.drop('Age', axis=1)
y = df_known_age['Age']

# Train-test split (optional for evaluation)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

# Predict missing Age values
x_missing = df_unknown_age.drop('Age', axis=1)
predicted_ages = model.predict(x_missing)

# Impute missing ages in original DataFrame
df.loc[df['Age'].isna(), 'Age'] = predicted_ages


In [145]:
# dropping null value presented in the embarked
df.dropna(subset='Embarked',inplace=True)

In [146]:
# impute the null value presented in the cabin
df.drop(['Name','Ticket'],axis=1,inplace=True)
df.drop(['Cabin'],axis=1,inplace=True)

In [147]:
# label encodeing the categorical column
df.head()
# Encode 'Embarked'
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])
# Encode 'Sex'
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])

In [148]:
#Setting x and y
X= df.drop('Survived',axis=1)
y=df['Survived']
len(X)
len(y)

889

In [149]:
# doing the cross validation
kf=KFold(n_splits=5,shuffle=True,random_state=43)
random_model=RandomForestClassifier()

# doing hyperparameter Tuining
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],

    'min_samples_split': [2, 5]
}
grid = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=kf,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

grid.fit(X, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=43, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='neg_mean_squared_error', verbose=1)

In [150]:
# Best estimator
best_model = grid.best_estimator_
print("Best Params:", grid.best_params_)

Best Params: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50}


In [151]:
cv_scores = cross_val_score(best_model, X, y, cv=kf, scoring='accuracy')
print("Cross-validated accuracy scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())

Cross-validated accuracy scores: [0.85955056 0.86516854 0.83707865 0.83146067 0.84745763]
Mean accuracy: 0.8481432108169873


In [155]:
# doing the randomized search cv

# doing the cross validation
kf=KFold(n_splits=5,shuffle=True,random_state=43)
random_model=RandomForestClassifier()

# doing hyperparameter Tuining
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
random_grid = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_grid,
    n_iter=10,
    cv=kf,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

random_grid.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=43, shuffle=True),
                   estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [50, 100, 150]},
                   scoring='neg_mean_squared_error', verbose=1)

In [157]:
# finding the best model given by randomized search
# Best estimator
best_model_random = random_grid.best_estimator_
print("Best Params:", random_grid.best_params_)

Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 10}


In [158]:
# evaluating the cv for this
cv_scores = cross_val_score(best_model_random, X, y, cv=kf, scoring='accuracy')
print("Cross-validated accuracy scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())

Cross-validated accuracy scores: [0.85955056 0.86516854 0.83707865 0.83146067 0.84745763]
Mean accuracy: 0.8481432108169873


## use the logistic regression

In [170]:
param_grid = {
    'penalty': ['l1', 'l2', None],
    'C': [0.01, 0.1, 1, 10],
    'solver': ['saga']

}

random_model = LogisticRegression(random_state=42, max_iter=1000)

logistic_grid = RandomizedSearchCV(
    estimator=random_model,
    param_distributions=param_grid,
    n_iter=10,
    cv=kf,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

random_grid.fit(X, y)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RandomizedSearchCV(cv=KFold(n_splits=5, random_state=43, shuffle=True),
                   estimator=LogisticRegression(max_iter=1000, random_state=42),
                   n_jobs=-1,
                   param_distributions={'C': [0.01, 0.1, 1, 10],
                                        'l1_ratio': [0.0, 0.5, 1.0],
                                        'penalty': ['l1', 'l2', 'elasticnet',
                                                    None],
                                        'solver': ['saga']},
                   scoring='accuracy', verbose=1)

In [171]:
best_model_Logistic = random_grid.best_estimator_
print("Best Params:", random_grid.best_params_)


Best Params: {'solver': 'saga', 'penalty': None, 'l1_ratio': 0.0, 'C': 10}


In [172]:
# evaluating the cv for this
cv_scores = cross_val_score(best_model_Logistic, X, y, cv=kf, scoring='accuracy')

print("Cross-validated accuracy scores:", cv_scores)
print("Mean accuracy:", cv_scores.mean())

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The

Cross-validated accuracy scores: [0.69662921 0.73033708 0.6741573  0.65168539 0.71186441]
Mean accuracy: 0.6929346791087412


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=None)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1207: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
